In [1]:
import pandas as pd
import numpy as np

In [2]:
logs = pd.read_csv('log.xls', header=None)
users = pd.read_csv('users.csv', encoding='koi8_r', sep='\t')

In [3]:
logs = logs.rename(columns={0:'user_id', 1:'time', 2:'bet', 3:'win'})
users = users.rename(columns={'Юзверь':'user_id', 'мейл':'email', 'Гео':'city'})

In [4]:
logs['time'].isna().sum()

15

In [5]:
logs['time'] = logs['time'].apply(lambda x: x if pd.isna(x) else str(x)[1:])

In [6]:
logs['time'] = logs['time'].apply(lambda x: x if pd.isna(x) else pd.to_datetime(x))

In [7]:
logs['time'].max()

Timestamp('2019-04-20 18:10:07')

In [8]:
logs['minute'] = logs['time'].dt.minute

In [9]:
logs['month'] = logs['time'].dt.month

In [10]:
logs['month'].value_counts()

1.0    292
3.0    264
2.0    259
4.0    170
Name: month, dtype: int64

In [11]:
logs['dayofweek'] = logs['time'].dt.dayofweek

In [12]:
logs[logs['dayofweek'] >= 5]

,user_id,time,bet,win,minute,month,dayofweek
39,Запись пользователя № - user_916,2019-01-05 01:54:07,NaN,NaN,54.0,1.0,5.0
40,Запись пользователя № - user_942,2019-01-05 10:44:56,NaN,NaN,44.0,1.0,5.0
41,Запись пользователя № - user_954,2019-01-05 13:13:21,NaN,NaN,13.0,1.0,5.0
42,Запись пользователя № - user_985,2019-01-05 14:36:15,NaN,NaN,36.0,1.0,5.0
43,Запись пользователя № - user_938,2019-01-05 18:09:15,NaN,NaN,9.0,1.0,5.0
...,...,...,...,...,...,...,...
991,Запись пользователя № - user_965,2019-04-20 12:55:41,800.0,6927.0,55.0,4.0,5.0
992,Запись пользователя № - user_967,2019-04-20 14:59:36,10154.0,NaN,59.0,4.0,5.0
993,Запись пользователя № - user_973,2019-04-20 17:09:56,10254.0,NaN,9.0,4.0,5.0
994,Запись пользователя № - user_977,2019-04-20 18:10:07,10354.0,NaN,10.0,4.0,5.0


In [13]:
#logs = logs.dropna(subset=['time'])

In [13]:
def time_of_day(x):
    if 0 <= x.hour <= 5:
        return 'night' 
    if 6 <= x.hour <= 11:
        return 'morning'
    if 12 <= x.hour <= 17:
        return 'day'
    if 18 <= x.hour <= 23:
        return 'evening'
    else:
        return x 
    

In [14]:
logs['time_of_day'] = logs['time'].apply(time_of_day)

In [15]:
logs['time_of_day'].value_counts()

night      265
morning    253
day        240
evening    227
Name: time_of_day, dtype: int64

In [16]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
user_id        1000 non-null object
time           985 non-null datetime64[ns]
bet            485 non-null float64
win            138 non-null float64
minute         985 non-null float64
month          985 non-null float64
dayofweek      985 non-null float64
time_of_day    985 non-null object
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 62.6+ KB


In [17]:
logs['bet'] = logs['bet'].fillna(0)

In [18]:
logs['bet'].value_counts()

0.0          515
500.0         50
100.0         48
300.0         42
800.0         42
200.0         40
400.0         40
700.0         38
600.0         37
1000.0        37
900.0         30
9754.0        10
10554.0       10
10154.0        7
10254.0        7
10454.0        6
9954.0         6
10754.0        5
10654.0        5
10354.0        4
10054.0        4
9854.0         2
5000.0         1
27000.0        1
9876.0         1
156789.0       1
12945.0        1
12548.0        1
145732.0       1
1678321.0      1
98753.0        1
123981.0       1
104540.0       1
8700.0         1
950.0          1
8734.0         1
7650.0         1
Name: bet, dtype: int64

In [19]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
user_id        1000 non-null object
time           985 non-null datetime64[ns]
bet            1000 non-null float64
win            138 non-null float64
minute         985 non-null float64
month          985 non-null float64
dayofweek      985 non-null float64
time_of_day    985 non-null object
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 62.6+ KB


In [20]:
#-------

In [27]:
# Приведем признак user_id к одному формату в обоих датасетах  
users.user_id = users.user_id.apply(lambda x: x.lower())  
# Избавимся от ошибок в user_id  
logs = logs[logs.user_id != '#error']  
logs.user_id = logs.user_id.str.split(' - ').apply(lambda x: x[1])  

In [36]:
logs.win = logs.win.fillna(0)
logs.bet = logs.bet.fillna(0)
logs['net'] = logs.win - logs.bet

In [38]:
df = pd.merge(users, logs, on='user_id') 

In [42]:
df.groupby('user_id').net.sum().median()

1986.0

In [44]:
df.groupby('user_id').count()

,email,city,time,bet,win,minute,month,dayofweek,time_of_day,net
user_id,,,,,,,,,,
user_1000,12,12,12,12,12,12,12,12,12,12
user_900,14,14,14,14,14,14,14,14,14,14
user_901,8,8,8,8,8,8,8,8,8,8
user_902,8,0,8,8,8,8,8,8,8,8
user_903,9,9,9,9,9,9,9,9,9,9
...,...,...,...,...,...,...,...,...,...,...
user_995,10,10,10,10,10,10,10,10,10,10
user_996,5,5,5,5,5,5,5,5,5,5
user_997,6,6,6,6,6,6,6,6,6,6


In [47]:
df['bet_counter'] = df['bet'].apply(lambda x: 0 if x == 0 else 1)

In [48]:
df

,user_id,email,city,time,bet,win,minute,month,dayofweek,time_of_day,net,bet_counter
0,user_943,Accumanst@gmail.com,Ижевск,2019-01-06 21:06:57,0.0,0.0,6.0,1.0,6.0,evening,0.0,0
1,user_943,Accumanst@gmail.com,Ижевск,2019-01-10 12:16:22,100.0,0.0,16.0,1.0,3.0,day,-100.0,1
2,user_943,Accumanst@gmail.com,Ижевск,2019-01-27 01:18:47,100.0,0.0,18.0,1.0,6.0,night,-100.0,1
3,user_943,Accumanst@gmail.com,Ижевск,2019-02-10 11:09:24,100.0,5456.0,9.0,2.0,6.0,morning,5356.0,1
4,user_943,Accumanst@gmail.com,Ижевск,2019-02-12 15:09:38,300.0,5245.0,9.0,2.0,1.0,day,4945.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
970,user_965,Visuareda@yahoo.com,Воронеж,2019-04-20 12:55:41,800.0,6927.0,55.0,4.0,5.0,day,6127.0,1
971,user_921,Aavast@ya.ru,Ижевск,2019-01-07 01:05:59,0.0,0.0,5.0,1.0,0.0,night,0.0,0
972,user_921,Aavast@ya.ru,Ижевск,2019-02-05 14:08:57,0.0,0.0,8.0,2.0,1.0,day,0.0,0
973,user_921,Aavast@ya.ru,Ижевск,2019-02-09 23:20:21,400.0,6067.0,20.0,2.0,5.0,evening,5667.0,1


In [51]:
bet_sum = df.groupby('user_id').bet_counter.sum().reset_index()

In [ ]:
df.groupby('user_id').bet_counter.sum().reset_index()

In [58]:
bet_sum.mean()

bet_counter    4.7
dtype: float64

In [60]:
df[(df.user_id == 'user_943') & df.bet == 0].time.min()

Timestamp('2019-01-01 04:37:25')

In [63]:
import matplotlib

In [81]:
delta = []

for i in df.user_id.unique():
    first_viz = df[(df.user_id == i) & (df.bet == 0)].time.min()
    bet_viz = df[(df.user_id == i) & (df.bet != 0)].time.min()
    delta_days = bet_viz - first_viz
    delta.append(delta_days)

In [82]:
pd.Series(delta).median()

Timedelta('49 days 23:25:56.500000')

In [75]:
df.user_id.nunique()

100

In [79]:
df_first_appearance = pd.DataFrame(df.groupby('user_id')['time'].first()).reset_index()
df_first_bet = df[df['bet'] > 0].groupby('user_id')['time'].first().reset_index()
df_merged = pd.merge(df_first_appearance, df_first_bet, on='user_id')
df_merged['time_diff'] = df_merged['time_y'] - df_merged['time_x']
df_merged['day_diff'] = df_merged['time_diff'].dt.days
df_merged['day_diff'].mean()

46.59

In [86]:
df.groupby('city').win.sum().reset_index().sort_values('win')

,city,win
11,Тюмень,4701.0
2,Екатеринбург,36682.0
10,Ставрополь,46003.0
5,Краснодар,62718.0
12,Хабаровск,65459.0
8,Пермь,67734.0
0,Арзангелтск,74375.0
6,Красноярск,84767.0
3,Ижевск,84895.0
13,Ярославль,97441.0


In [92]:
df[df['bet'] != 0].groupby('city').bet.mean().min()

486.95652173913044

In [96]:
df[df['bet'] != 0].groupby('city').bet.mean().max() / df[df['bet'] != 0].groupby('city').bet.mean().min()

127.81602335164835

---------

In [119]:
log = pd.read_csv('log.xls', header=None)

In [120]:
log = log.rename(columns={0:'user_id', 1:'time', 2:'bet', 3:'win'})

In [121]:
log.win = log.win.fillna(0)
log.bet = log.bet.fillna(0)
log['net'] = log.win - log.bet

In [125]:
log[log.bet > 0].shape[0] / 1000

0.485

In [126]:
log

,user_id,time,bet,win,net
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,0.0,0.0,0.0
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,0.0,0.0,0.0
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,0.0,0.0,0.0
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,0.0,0.0,0.0
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,0.0,0.0,0.0
...,...,...,...,...,...
995,Запись пользователя № - user_984,[2019-04-20 9:59:58,9754.0,0.0,-9754.0
996,#error,NaN,10054.0,29265.0,19211.0
997,#error,NaN,10454.0,0.0,-10454.0
998,#error,NaN,1000.0,0.0,-1000.0


In [131]:
log[log.net < 0]['net'].mean()

-3372.743515850144

In [132]:
log[(log.bet > 0) & (log.net > 0)]

,user_id,time,bet,win,net
14,Запись пользователя № - user_917,[2019-01-02 8:57:36,145732.0,1987653.0,1841921.0
29,Запись пользователя № - user_942,[2019-01-04 13:59:42,1678321.0,9876543.0,8198222.0
151,Запись пользователя № - user_982,[2019-01-16 21:54:22,100.0,4749.0,4649.0
189,Запись пользователя № - user_964,[2019-01-21 18:34:44,200.0,4667.0,4467.0
205,Запись пользователя № - user_931,[2019-01-22 5:26:59,300.0,4319.0,4019.0
...,...,...,...,...,...
981,#error,NaN,800.0,7035.0,6235.0
986,#error,NaN,10454.0,29972.0,19518.0
987,#error,NaN,10554.0,31634.0,21080.0
991,Запись пользователя № - user_965,[2019-04-20 12:55:41,800.0,6927.0,6127.0


In [133]:
log[(log.bet > 0) & (log.net < 0)]

,user_id,time,bet,win,net
5,Запись пользователя № - user_917,[2019-01-01 23:34:55,156789.0,0.0,-156789.0
19,Запись пользователя № - user_942,[2019-01-03 14:03:21,98753.0,0.0,-98753.0
67,Запись пользователя № - user_987,[2019-01-07 21:46:13,5000.0,0.0,-5000.0
81,Запись пользователя № - user_910,[2019-01-08 21:14:57,123981.0,0.0,-123981.0
92,Запись пользователя № - user_964,[2019-01-09 3:05:33,100.0,0.0,-100.0
...,...,...,...,...,...
994,Запись пользователя № - user_977,[2019-04-20 18:10:07,10354.0,0.0,-10354.0
995,Запись пользователя № - user_984,[2019-04-20 9:59:58,9754.0,0.0,-9754.0
997,#error,NaN,10454.0,0.0,-10454.0
998,#error,NaN,1000.0,0.0,-1000.0


In [136]:
min_bet = log[log.bet > 0]['bet'].min()

In [139]:
    min_bet_amount = log[log.bet == min_bet].shape[0]

48

In [143]:
log.bet.value_counts().index.min()

0.0

In [153]:
users

,user_id,email,city
0,User_943,Accumanst@gmail.com,Ижевск
1,User_908,Advismowr@mail.ru,Ижевск
2,User_962,Anachso@ukr.net,Краснодар
3,User_973,Antecia@inbox.ru,Пермь
4,User_902,Balliaryva@ukr.net,NaN
...,...,...,...
95,User_959,UpdatesCurious@yahoo.com,Тюмень
96,User_901,V2artierso@mail.ru,Арзангелтск
97,User_970,Vashoterlo@bk.ru,Воронеж
98,User_965,Visuareda@yahoo.com,Воронеж


In [151]:
pd.merge(log, users, on='user_id')

,user_id,time,bet,win,email,city


In [81]:
def fillna_win(row):
    
    bet = row['bet']
    win = row['win']
    
    if pd.isna(win):
        if pd.isna(bet):
            return 0
        else:
            delta = 0 - bet
            return delta
    else:
        delta = win - bet
        return delta

In [82]:
new_win = log.apply(lambda row: fillna_win(row), axis=1)

In [83]:
new_win

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
        ...   
995    -9754.0
996    19211.0
997   -10454.0
998    -1000.0
999   -10754.0
Length: 1000, dtype: float64

In [84]:
log['win'] = new_win  

In [85]:
log

,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,0.0
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,0.0
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,0.0
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,0.0
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,0.0
...,...,...,...,...
995,Запись пользователя № - user_984,[2019-04-20 9:59:58,9754.0,-9754.0
996,#error,NaN,10054.0,19211.0
997,#error,NaN,10454.0,-10454.0
998,#error,NaN,1000.0,-1000.0


In [80]:
log[log.win > 0]

,user_id,time,bet,win
14,Запись пользователя № - user_917,[2019-01-02 8:57:36,145732.0,1987653.0
29,Запись пользователя № - user_942,[2019-01-04 13:59:42,1678321.0,9876543.0
151,Запись пользователя № - user_982,[2019-01-16 21:54:22,100.0,4749.0
189,Запись пользователя № - user_964,[2019-01-21 18:34:44,200.0,4667.0
205,Запись пользователя № - user_931,[2019-01-22 5:26:59,300.0,4319.0
...,...,...,...,...
981,#error,NaN,800.0,7035.0
986,#error,NaN,10454.0,29972.0
987,#error,NaN,10554.0,31634.0
991,Запись пользователя № - user_965,[2019-04-20 12:55:41,800.0,6927.0


In [96]:
log[log.win > 0]['win'].mean()

80253.33333333333

In [98]:
round(log[log.win > 0]['win'].median(), 0)

5347.0

In [100]:
log.bet.mean()

6785.738144329897

In [102]:
log.bet.sum() / log.bet.dropna().shape[0]

6785.738144329897

In [103]:
np.mean(log.bet)

6785.738144329897

In [104]:
log['bet'].dropna().mean()


6785.738144329897

In [107]:
log[log.bet > 0].shape[0] / log[log.win > 0].shape[0]

3.5144927536231885